In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['label'] = 0  # 0 = Fake
true['label'] = 1  # 1 = True


In [6]:
data = pd.concat([fake, true], axis=0)


In [7]:
data.sample(10)

,title,text,subject,date,label
16281,Plans to hold delayed Kenya vote postponed in ...,NAIROBI (Reuters) - Kenya s election commissio...,worldnews,"October 27, 2017",1
13425,"EU, U.N., African leaders draw up emergency pl...","ABIDJAN (Reuters) - The European Union, United...",worldnews,"November 30, 2017",1
19896,Pakistan army pushed political role for milita...,"LAHORE, Pakistan (Reuters) - A new Pakistani p...",worldnews,"September 16, 2017",1
15964,“DIRTY JOBS” HOST MIKE ROWE Absolutely Destroy...,Dirty Jobs host Mike Rowe went off on a guy ...,Government News,"Aug 23, 2017",0
11782,IT’S NOT A “MUSLIM BAN” You Big Dummies! [VIDEO],,politics,"Jan 29, 2017",0
3366,Trump Tells China To Keep Our Navy Drone And ...,Donald Trump just proved once again that he is...,News,"December 17, 2016",0
21793,BALTIMORE POLICE UNION WANTS AN INDEPENDANT PR...,The Police Union came out almost immediately a...,left-news,"May 1, 2015",0
17590,NBC MANAGEMENT “Protected The Sh*t Out of Him”...,NBC was called out by conservative news outlet...,left-news,"Nov 29, 2017",0
21340,Britain will not pay 'a penny more' than it th...,LONDON (Reuters) - Britain will pay not a pen...,worldnews,"August 25, 2017",1
9163,"Abortion providers, opponents brace for U.S. h...",WASHINGTON (Reuters) - As she waits for the U....,politicsNews,"June 12, 2016",1


In [8]:
data = data.drop(["title","subject","date"],axis=1)

In [11]:
data = data.sample(frac=1)
data.reset_index(drop=True, inplace=True)


In [17]:
data.sample(5)

,text,label
3855,(Reuters) - Highlights of the day for U.S. Pre...,1
40200,(Reuters) - New Jersey Governor Chris Christie...,1
39853,"After 20 years of corruption, millions of doll...",0
35700,WASHINGTON (Reuters) - The U.S. policy of tryi...,1
14681,TOKYO (Reuters) - U.S. President Donald Trump ...,1


In [18]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [19]:
data["text"] = data["text"].apply(clean_text)


In [20]:
x=data["text"]
y=data["label"]
xtrain, xtest, ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [21]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)


In [23]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)


LogisticRegression()

In [24]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9879732739420936

In [25]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5860
           1       0.99      0.99      0.99      5365

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [26]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']